In [1]:
#remove cell visibility
from IPython.display import HTML
tag = HTML('''<script>
code_show=true; 
function code_toggle() {
    if (code_show){
        $('div.input').hide()
    } else {
        $('div.input').show()
    }
    code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
Toggle cell visibility <a href="javascript:code_toggle()">here</a>.''')
display(tag)

In [2]:
%matplotlib inline
import control
import numpy
import sympy as sym
from IPython.display import display, Markdown
import ipywidgets as widgets
import matplotlib.pyplot as plt


#print a matrix latex-like
def bmatrix(a):
     """Returns a LaTeX bmatrix - by Damir Arbula (ICCT project)

     :a: numpy array
     :returns: LaTeX bmatrix as a string
     """
     if len(a.shape) > 2:
         raise ValueError('bmatrix can at most display two dimensions')
     lines = str(a).replace('[', '').replace(']', '').splitlines()
     rv = [r'\begin{bmatrix}']
     rv += ['  ' + ' & '.join(l.split()) + r'\\' for l in lines]
     rv +=  [r'\end{bmatrix}']
     return '\n'.join(rv)


# Display formatted matrix: 
def vmatrix(a):
    if len(a.shape) > 2:
         raise ValueError('bmatrix can at most display two dimensions')
    lines = str(a).replace('[', '').replace(']', '').splitlines()
    rv = [r'\begin{vmatrix}']
    rv += ['  ' + ' & '.join(l.split()) + r'\\' for l in lines]
    rv +=  [r'\end{vmatrix}']
    return '\n'.join(rv)


#matrixWidget is a matrix looking widget built with a VBox of HBox(es) that returns a numPy array as value !
class matrixWidget(widgets.VBox):
    def updateM(self,change):
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.M_[irow,icol] = self.children[irow].children[icol].value
                #print(self.M_[irow,icol])
        self.value = self.M_

    def dummychangecallback(self,change):
        pass
    
    
    def __init__(self,n,m):
        self.n = n
        self.m = m
        self.M_ = numpy.matrix(numpy.zeros((self.n,self.m)))
        self.value = self.M_
        widgets.VBox.__init__(self,
                             children = [
                                 widgets.HBox(children = 
                                              [widgets.FloatText(value=0.0, layout=widgets.Layout(width='90px')) for i in range(m)]
                                             ) 
                                 for j in range(n)
                             ])
        
        #fill in widgets and tell interact to call updateM each time a children changes value
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].value = self.M_[irow,icol]
                self.children[irow].children[icol].observe(self.updateM, names='value')
        #value = Unicode('example@example.com', help="The email value.").tag(sync=True)
        self.observe(self.updateM, names='value', type= 'All')
        
    def setM(self, newM):
        #disable callbacks, change values, and reenable
        self.unobserve(self.updateM, names='value', type= 'All')
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].unobserve(self.updateM, names='value')
        self.M_ = newM
        self.value = self.M_
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].value = self.M_[irow,icol]
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].observe(self.updateM, names='value')
        self.observe(self.updateM, names='value', type= 'All')        

                #self.children[irow].children[icol].observe(self.updateM, names='value')

             
#overlaod class for state space systems that DO NOT remove "useless" states (what "professor" of automatic control would do this?)
class sss(control.StateSpace):
    def __init__(self,*args):
        #call base class init constructor
        control.StateSpace.__init__(self,*args)
    #disable function below in base class
    def _remove_useless_states(self):
        pass

## Controllo della traiettoria di un aereo a terra

La dinamica rotazionale di un aereo che si muove a terra può essere rappresentata come:

$$
J_z\ddot{\psi} = bF_1\delta + F_2\dot{\psi} \, , 
$$

dove $J_z = 11067000$ kg$\text{m}^2$, $b = 15$, $F_1 = 35000000$ Nm, $F_2 = 500000$ kg$\text{m}^2$ / $\text{s}$, $\psi$ è l'angolo di rotazione (in rad), o angolo di imbardata, rispetto all'asse verticale, e $\delta$ è l'angolo di sterzata della ruota anteriore (in rad). Quando l'aereo sta seguendo una linea retta con velocità lineare longitudinale $V$ (in m / s), la velocità laterale dell'aeromobile $V_y$ (in m / s) è approssimativamente linearmente proporzionale all'angolo di imbardata: $V_y = \dot{p_y} = V\psi$.

L'obiettivo è progettare un regolatore per la posizione laterale dell'aereo $p_y$, con velocità longitudinale $V$ impostata a 35 km/h, utilizzando l'angolo di sterzo della ruota anteriore $\delta$ come input del sistema secondo le seguenti specifiche:
- tempo di assestamento al 5% inferiore a 4 secondi;
- errore di regime nullo in risposta a una richiesta di posizione laterale desiderata;
- overshoot minimo o nullo;
- l'angolo di sterzata non deve superare $\pm8$ gradi quando si segue un gradino di posizione laterale di 5 metri.

Le equazioni dinamiche in forma di stato sono:

\begin{cases}
    \dot{x} = \begin{bmatrix} \frac{F_2}{J_z} & 0 & 0 \\ 1 & 0 & 0 \\ 0 & V & 0 \end{bmatrix}x + \begin{bmatrix} \frac{bF_1}{J_z} \\ 0 \\ 0 \end{bmatrix}u \\
    y = \begin{bmatrix} 0 & 0 & 1 \end{bmatrix}x \, ,
\end{cases}

dove $x=\begin{bmatrix} x_1 & x_2 & x_3 \end{bmatrix}^T = \begin{bmatrix} \dot{\psi} & \psi & p_y \end{bmatrix}^T$ e $u=\delta$.

I poli del sistema sono $0$, $0$ e $\frac{F_2}{J_z} \simeq 0.045$, quindi il sistema è instabile.

### Design del regolatore
#### Design del controller

Per soddisfare i requisiti di errore di regime nullo si aggiunge un nuovo stato:
$$
\dot{x_4} = p_y-y_d = x_3 - y_d
$$
Il sistema aumentato risultante è quindi:

\begin{cases}
    \dot{x_a} = \begin{bmatrix} \frac{F_2}{J_z} & 0 & 0 & 0 \\ 1 & 0 & 0 & 0 \\ 0 & V & 0 & 0 \\ 0 & 0 & 1 & 0 \end{bmatrix}x_a + \begin{bmatrix} \frac{bF_1}{J_z} & 0 \\ 0 & 0 \\ 0 & 0 \\ 0 & -1 \end{bmatrix}\begin{bmatrix} u \\ y_d \end{bmatrix} \\
    y_a = \begin{bmatrix} 0 & 0 & 1 & 0 \\ 0 & 0 & 0 & 1 \end{bmatrix}x_a,
\end{cases}

dove vengono aggiunti $x_a = \begin{bmatrix} x_1 & x_2 & x_3 & x_4 \end{bmatrix}^T$ e la seconda uscita per mantenere l'osservabilità. Il sistema rimane controllabile con l'ingresso $u$ e quindi si può progettare il feedback dello stato con questo ingresso. Una possibile soluzione è posizionare tutti i poli in $-2$.

#### Design dell'osservatore

Anche se gli stati $x_3$ e $x_4$ possono essere ottenuti dalle misure ed è necessario stimare solo $x_2$ e $x_3$, è conveniente lavorare con il sistema $4\times4$ completo e progettare un osservatore di stato di ordine 4 con tutti i poli in $-10$.

### Come usare questo notebook?
- Verifica se i requisiti sono soddisfatti in presenza di errori nella stima dello stato iniziale.

In [3]:
# Preparatory cell

X0 = numpy.matrix('0.0; 0.0; 0.0; 0.0')
K = numpy.matrix([0,0,0,0])
L = numpy.matrix([[0,0],[0,0],[0,0],[0,0]])

X0w = matrixWidget(4,1)
X0w.setM(X0)
Kw = matrixWidget(1,4)
Kw.setM(K)
Lw = matrixWidget(4,2)
Lw.setM(L)


eig1c = matrixWidget(1,1)
eig2c = matrixWidget(2,1)
eig3c = matrixWidget(1,1)
eig4c = matrixWidget(2,1)
eig1c.setM(numpy.matrix([-2.])) 
eig2c.setM(numpy.matrix([[-2.],[-0.]]))
eig3c.setM(numpy.matrix([-2.]))
eig4c.setM(numpy.matrix([[-2.],[-0.]]))

eig1o = matrixWidget(1,1)
eig2o = matrixWidget(2,1)
eig3o = matrixWidget(1,1)
eig4o = matrixWidget(2,1)
eig1o.setM(numpy.matrix([-10.])) 
eig2o.setM(numpy.matrix([[-10.],[0.]]))
eig3o.setM(numpy.matrix([-10.]))
eig4o.setM(numpy.matrix([[-10.],[0.]]))

In [4]:
# Misc

#create dummy widget 
DW = widgets.FloatText(layout=widgets.Layout(width='0px', height='0px'))

#create button widget
START = widgets.Button(
    description='Test',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Test',
    icon='check'
)
                       
def on_start_button_clicked(b):
    #This is a workaround to have intreactive_output call the callback:
    #   force the value of the dummy widget to change
    if DW.value> 0 :
        DW.value = -1
    else: 
        DW.value = 1
    pass
START.on_click(on_start_button_clicked)

# Define type of method 
selm = widgets.Dropdown(
    options= [('Imposta K e L','Set K and L'), ('Imposta gli autovalori','Set the eigenvalues')],
    value= 'Set the eigenvalues',
    description='',
    disabled=False
)

# Define the number of complex eigenvalues
selec = widgets.Dropdown(
    options= [('0 autovalori complessi','0 complex eigenvalues'), ('2 autovalori complessi','2 complex eigenvalues'), ('4 autovalori complessi','4 complex eigenvalues')],
    value= '0 complex eigenvalues',
    description='Autov. controller:',
    style = {'description_width': 'initial'},
    disabled=False
)
seleo = widgets.Dropdown(
    options= [('0 autovalori complessi','0 complex eigenvalues'), ('2 autovalori complessi','2 complex eigenvalues'), ('4 autovalori complessi','4 complex eigenvalues')],
    value= '0 complex eigenvalues',
    description='Autov. osservatore:',
    style = {'description_width': 'initial'},
    disabled=False
)

#define type of ipout 
selu = widgets.Dropdown(
    options=[('impulso','impulse'), ('gradino','step'), ('sinusoide','sinusoid'), ('onda quadra','square wave')],
    value='step',
    description='Riferimento:',
    style = {'description_width': 'initial'},
    disabled=False
)
# Define the values of the input
u = widgets.FloatSlider(
    value=5,
    min=0,
    max=10,
    step=0.1,
    description='Riferimento [m]:',
    style = {'description_width': 'initial'},
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)
v = widgets.FloatSlider(
    value=9.72,
    min=1,
    max=20,
    step=0.1,
    description=r'$V$ [m/s]:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)
period = widgets.FloatSlider(
    value=0.5,
    min=0.001,
    max=10,
    step=0.001,
    description='Periodo: ',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.3f',
)

simTime = widgets.FloatText(
    value=5,
    description='',
#     description='Tempo di simulazione (s):',
#     style = {'description_width': 'initial'},
    disabled=False
)

In [5]:
# Support functions

def eigen_choice(selec,seleo):
    if selec == '0 complex eigenvalues':
        eig1c.children[0].children[0].disabled = False
        eig2c.children[1].children[0].disabled = True
        eig3c.children[0].children[0].disabled = False
        eig4c.children[0].children[0].disabled = False
        eig4c.children[1].children[0].disabled = True
        eigc = 0
    if seleo == '0 complex eigenvalues':
        eig1o.children[0].children[0].disabled = False
        eig2o.children[1].children[0].disabled = True
        eig3o.children[0].children[0].disabled = False
        eig4o.children[0].children[0].disabled = False
        eig4o.children[1].children[0].disabled = True
        eigo = 0
    if selec == '2 complex eigenvalues':
        eig1c.children[0].children[0].disabled = False
        eig2c.children[1].children[0].disabled = False
        eig3c.children[0].children[0].disabled = False
        eig4c.children[0].children[0].disabled = True
        eig4c.children[1].children[0].disabled = True
        eigc = 2
    if seleo == '2 complex eigenvalues':
        eig1o.children[0].children[0].disabled = False
        eig2o.children[1].children[0].disabled = False
        eig3o.children[0].children[0].disabled = False
        eig4o.children[0].children[0].disabled = True
        eig4o.children[1].children[0].disabled = True
        eigo = 2
    if selec == '4 complex eigenvalues':
        eig1c.children[0].children[0].disabled = True
        eig2c.children[1].children[0].disabled = False
        eig3c.children[0].children[0].disabled = True
        eig4c.children[0].children[0].disabled = False
        eig4c.children[1].children[0].disabled = False
        eigc = 4
    if seleo == '4 complex eigenvalues':
        eig1o.children[0].children[0].disabled = True
        eig2o.children[1].children[0].disabled = False
        eig3o.children[0].children[0].disabled = True
        eig4o.children[0].children[0].disabled = False
        eig4o.children[1].children[0].disabled = False
        eigo = 4
    return eigc, eigo

def method_choice(selm):
    if selm == 'Set K and L':
        method = 1
        selec.disabled = True
        seleo.disabled = True
    if selm == 'Set the eigenvalues':
        method = 2
        selec.disabled = False
        seleo.disabled = False
    return method

In [6]:
F1 = 35000000
F2 = 500000
b = 15
V = 35/3.6
Jz = 11067000

A = numpy.matrix([[F2/Jz, 0, 0, 0],
                  [1, 0, 0, 0],
                  [0, V, 0, 0],
                  [0, 0, 1, 0]])
Bu = numpy.matrix([[b*F1/Jz],[0],[0],[0]])
Bref = numpy.matrix([[0],[0],[0],[-1]])
C = numpy.matrix([[0,0,1,0],[0,0,0,1]])

def main_callback2(v, X0w, K, L, eig1c, eig2c, eig3c, eig4c, eig1o, eig2o, eig3o, eig4o, u, period, selm, selec, seleo, selu, simTime, DW):
    eigc, eigo = eigen_choice(selec,seleo)
    method = method_choice(selm)
    
    A = numpy.matrix([[F2/Jz, 0, 0, 0],
                  [1, 0, 0, 0],
                  [0, v, 0, 0],
                  [0, 0, 1, 0]])
    
    if method == 1:
        solc = numpy.linalg.eig(A-Bu*K)
        solo = numpy.linalg.eig(A-L*C)
    if method == 2:
        #for better numerical stability of place
        if eig1c[0,0]==eig2c[0,0] or eig1c[0,0]==eig3c[0,0] or eig1c[0,0]==eig4c[0,0]:
            eig1c[0,0] *= 1.01
        if eig2c[0,0]==eig3c[0,0] or eig2c[0,0]==eig4c[0,0]:
            eig3c[0,0] *= 1.015
        if eig1o[0,0]==eig2o[0,0] or eig1o[0,0]==eig3o[0,0] or eig1o[0,0]==eig4o[0,0]:
            eig1o[0,0] *= 1.01
        if eig2o[0,0]==eig3o[0,0] or eig2o[0,0]==eig4o[0,0]:
            eig3o[0,0] *= 1.015
            
        if eigc == 0:
            K = control.acker(A, Bu, [eig1c[0,0], eig2c[0,0], eig3c[0,0], eig4c[0,0]])
            Kw.setM(K)
        if eigc == 2:
            K = control.acker(A, Bu, [eig3c[0,0],
                                      eig1c[0,0],
                                      numpy.complex(eig2c[0,0], eig2c[1,0]), 
                                      numpy.complex(eig2c[0,0],-eig2c[1,0])])
            Kw.setM(K)
        if eigc == 4:
            K = control.acker(A, Bu, [numpy.complex(eig4c[0,0], eig4c[1,0]), 
                                      numpy.complex(eig4c[0,0],-eig4c[1,0]),
                                      numpy.complex(eig2c[0,0], eig2c[1,0]), 
                                      numpy.complex(eig2c[0,0],-eig2c[1,0])])
            Kw.setM(K)
        if eigo == 0:
            L = control.place(A.T, C.T, [eig1o[0,0], eig2o[0,0], eig3o[0,0], eig4o[0,0]]).T
            Lw.setM(L)
        if eigo == 2:
            L = control.place(A.T, C.T, [eig3o[0,0],
                                         eig1o[0,0],
                                         numpy.complex(eig2o[0,0], eig2o[1,0]), 
                                         numpy.complex(eig2o[0,0],-eig2o[1,0])]).T
            Lw.setM(L)
        if eigo == 4:
            L = control.place(A.T, C.T, [numpy.complex(eig4o[0,0], eig4o[1,0]), 
                                      numpy.complex(eig4o[0,0],-eig4o[1,0]),
                                      numpy.complex(eig2o[0,0], eig2o[1,0]), 
                                      numpy.complex(eig2o[0,0],-eig2o[1,0])]).T
            Lw.setM(L)
            
    sys = sss(A,numpy.hstack((Bu,Bref)),[[0,0,1,0],[0,0,0,1],[0,0,0,0]],[[0,0],[0,0],[0,1]])
    syse = sss(A-L*C,numpy.hstack((Bu,Bref,L)),numpy.eye(4),numpy.zeros((4,4)))
    sysc = sss(0,[0,0,0,0],0,-K)
    sys_append = control.append(sys,syse,sysc)
    try:
        sys_CL = control.connect(sys_append,
                                 [[1,8],[3,8],[5,1],[6,2],[7,4],[8,5],[9,6],[10,7],[4,3]],
                                 [2],
                                 [1,8])
    except:
        sys_CL = control.connect(sys_append,
                                 [[1,8],[3,8],[5,1],[6,2],[7,4],[8,5],[9,6],[10,7],[4,3]],
                                 [2],
                                 [1,8])

    X0w1 = numpy.zeros((8,1))
    X0w1[4,0] = X0w[0,0]
    X0w1[5,0] = X0w[1,0]
    X0w1[6,0] = X0w[2,0]
    X0w1[7,0] = X0w[3,0]
    if simTime != 0:
        T = numpy.linspace(0, simTime, 10000)
    else:
        T = numpy.linspace(0, 1, 10000)
      
    if selu == 'impulse': #selu
        U = [0 for t in range(0,len(T))]
        U[0] = u
        T, yout, xout = control.forced_response(sys_CL,T,U,X0w1)
    if selu == 'step':
        U = [u for t in range(0,len(T))]
        T, yout, xout = control.forced_response(sys_CL,T,U,X0w1)
    if selu == 'sinusoid':
        U = u*numpy.sin(2*numpy.pi/period*T)
        T, yout, xout = control.forced_response(sys_CL,T,U,X0w1)
    if selu == 'square wave':
        U = u*numpy.sign(numpy.sin(2*numpy.pi/period*T))
        T, yout, xout = control.forced_response(sys_CL,T,U,X0w1)
    
    try:
        step_info_dict = control.step_info(sys_CL[0,0],SettlingTimeThreshold=0.05,T=T)
        print('Step info: \n\tTempo di salita =',step_info_dict['RiseTime'],'\n\tTempo di assestamento (5%) =',step_info_dict['SettlingTime'],'\n\tOvershoot (%)=',step_info_dict['Overshoot'])
        print('Massimo valore di u (% di 8 gradi)=', max(abs(yout[1]))/(8*numpy.pi/180)*100) 
    except:
        print("Errore nel calcolo delle step info.")
    
    fig = plt.figure(num='Simulation1', figsize=(14,12))
    
    fig.add_subplot(221)
    plt.title('Risposta dell\'uscita')
    plt.ylabel('Uscita')
    plt.plot(T,yout[0],T,U,'r--')
    plt.xlabel('$t$ [s]')
    plt.legend(['$y$','Riferimento'])
    plt.axvline(x=0,color='black',linewidth=0.8)
    plt.axhline(y=0,color='black',linewidth=0.8)
    plt.grid()
    
    fig.add_subplot(222)
    plt.title('Ingresso')
    plt.ylabel('$u$ [deg]')
    plt.plot(T,yout[1]*180/numpy.pi)
    plt.plot(T,[8 for i in range(len(T))],'r--')
    plt.plot(T,[-8 for i in range(len(T))],'r--')
    plt.xlabel('$t$ [s]')
    plt.axvline(x=0,color='black',linewidth=0.8)
    plt.axhline(y=0,color='black',linewidth=0.8)
    plt.grid()
    
    fig.add_subplot(223)
    plt.title('Risposta degli stati')
    plt.ylabel('Stati')
    plt.plot(T,xout[0],
             T,xout[1],
             T,xout[2],
             T,xout[3])
    plt.xlabel('$t$ [s]')
    plt.axvline(x=0,color='black',linewidth=0.8)
    plt.axhline(y=0,color='black',linewidth=0.8)
    plt.legend(['$x_{1}$','$x_{2}$','$x_{3}$','$x_{4}$'])
    plt.grid()
    
    fig.add_subplot(224)
    plt.title('Errori di stima')
    plt.ylabel('Errori')
    plt.plot(T,xout[4]-xout[0])
    plt.plot(T,xout[5]-xout[1])
    plt.plot(T,xout[6]-xout[2])
    plt.plot(T,xout[7]-xout[3])
    plt.xlabel('$t$ [s]')
    plt.axvline(x=0,color='black',linewidth=0.8)
    plt.axhline(y=0,color='black',linewidth=0.8)
    plt.legend(['$e_{1}$','$e_{2}$','$e_{3}$','$e_{4}$'])
    plt.grid()
    #plt.tight_layout()
   
alltogether2 = widgets.VBox([widgets.HBox([selm, 
                                          selec,
                                          seleo,
                                          selu]),
                            widgets.Label(' ',border=3),
                            widgets.HBox([widgets.HBox([widgets.Label('K:',border=3), Kw, 
                                                        widgets.Label('Autovalori:',border=3),
                                                        widgets.HBox([eig1c, 
                                                                      eig2c, 
                                                                      eig3c,
                                                                      eig4c])])]),
                            widgets.Label(' ',border=3),
                            widgets.HBox([widgets.VBox([widgets.HBox([widgets.Label('L:',border=3), Lw, widgets.Label(' ',border=3),
                                          widgets.Label(' ',border=3),
                                          widgets.Label('Autovalori:',border=3), 
                                          eig1o, 
                                          eig2o,
                                          eig3o, 
                                          eig4o,
                                          widgets.Label(' ',border=3),
                                          widgets.Label(' ',border=3),
                                          widgets.Label('X0 stim.:',border=3), X0w]),
                                          widgets.Label(' ',border=3),
                                                       widgets.HBox([
                                          widgets.VBox([widgets.Label('Tempo di simulazione [s]:',border=3)]),
                                          widgets.VBox([simTime])])]),
                                          widgets.Label(' ',border=3)]),
                            widgets.Label(' ',border=3),
                            widgets.HBox([u,
                                          v,
                                          period, 
                                          START])])
out2 = widgets.interactive_output(main_callback2, {'v':v, 'X0w':X0w, 'K':Kw, 'L':Lw,
                                                   'eig1c':eig1c, 'eig2c':eig2c, 'eig3c':eig3c, 'eig4c':eig4c, 
                                                   'eig1o':eig1o, 'eig2o':eig2o, 'eig3o':eig3o, 'eig4o':eig4o,  
                                                   'u':u, 'period':period, 'selm':selm, 'selec':selec, 'seleo':seleo, 'selu':selu, 'simTime':simTime, 'DW':DW})
out2.layout.height = '860px'
display(out2, alltogether2)

Output(layout=Layout(height='860px'))